In [1]:
!pip install transformers

In [2]:
!pip install -U PyPDF2
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 2.0 MB/s eta 0:00:00


In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [4]:
!pip install py7zr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.8/413.8 kB 27.8 MB/s eta 0:00:00


In [5]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset

# Load the SAMSum dataset
dataset = load_dataset('samsum')

# Limit the number of samples
num_train_samples = 500
num_val_samples = 200

# Slice training and validation datasets
train_dataset = dataset['train'].select(range(num_train_samples))
val_dataset = dataset['validation'].select(range(num_val_samples))

# Load tokenizer and add padding token
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
tokenizer.pad_token = tokenizer.eos_token  # Use the EOS token as padding token

# Preprocess function for SAMSum
def preprocess_function(examples):
    inputs = examples['dialogue']
    targets = examples['summary']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=150, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Tokenize and preprocess the dataset
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)

# Print dataset lengths
train_dataset_length = len(tokenized_train_dataset)
validation_dataset_length = len(tokenized_val_dataset)
print(f"Train dataset length: {train_dataset_length}")
print(f"Validation dataset length: {validation_dataset_length}")

# Define the data collator
def load_data_collator(tokenizer):
    return DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )

# Training function
def train(train_dataset, eval_dataset, model_name, output_dir, overwrite_output_dir, per_device_train_batch_size, num_train_epochs):
    model = GPT2LMHeadModel.from_pretrained(model_name)
    data_collator = load_data_collator(tokenizer)

    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=overwrite_output_dir,
        per_device_train_batch_size=per_device_train_batch_size,
        num_train_epochs=num_train_epochs,
        save_steps=100,
        logging_dir='./logs',
        logging_steps=10,  # Log every 10 steps
        evaluation_strategy="steps",
        eval_steps=10
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset
    )

    trainer.train()
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)

# Define paths and parameters
model_name = 'distilgpt2'
output_dir = '/content/custom_samsum'
overwrite_output_dir = True
per_device_train_batch_size = 4
num_train_epochs = 1

# Train the model
train(
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

...

Train dataset length: 500
Validation dataset length: 200


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
10,4.192300,3.865301
20,3.937100,3.591946
30,3.629100,3.464634
40,3.511500,3.399692
50,3.449600,3.340317
60,3.591000,3.292998
70,3.419800,3.257626
80,3.492600,3.225054
90,3.290700,3.205209
100,3.240300,3.194742


In [8]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

def load_model_and_tokenizer(model_path):
    # Load the fine-tuned model and tokenizer from the same path
    model = GPT2LMHeadModel.from_pretrained(model_path)
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    return model, tokenizer

def generate_text(model, tokenizer, prompt, max_new_tokens=150):
    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True, max_length=512, padding='max_length')

    # Generate text
    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],  # Provide attention mask
        max_new_tokens=max_new_tokens,  # Generate a specific number of new tokens
        num_return_sequences=1,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Define paths and prompt
model_path = '/content/custom_samsum'  # Path where the model and tokenizer are saved
prompt = "Summarize the following dialogue: [John decided to spend his weekend in nature. He packed a small bag with essentials: a water bottle, some snacks, and a warm jacket. Early Saturday morning, he drove to a nearby park known for its beautiful hiking trails. After a short drive, he arrived and started his hike. The weather was perfect—sunny with a gentle breeze. As he walked along the trail, he enjoyed the sounds of birds chirping and the rustling of leaves. By the end of the day, John felt refreshed and happy. It was a simple yet satisfying escape from his usual routine.]"

# Load the model and tokenizer
model, tokenizer = load_model_and_tokenizer(model_path)

# Generate and print text
generated_text = generate_text(model, tokenizer, prompt)
print(generated_text)


Summarize the following dialogue: [John decided to spend his weekend in nature. He packed a small bag with essentials: a water bottle, some snacks, and a warm jacket. Early Saturday morning, he drove to a nearby park known for its beautiful hiking trails. After a short drive, he arrived and started his hike. The weather was perfect—sunny with a gentle breeze. As he walked along the trail, he enjoyed the sounds of birds chirping and the rustling of leaves. By the end of the day, John felt refreshed and happy. It was a simple yet satisfying escape from his usual routine.]Photo: John
John then took the bus to a small town called Waukegan, where he was staying at the time. He got there in good standing. He made his way across to the town in the morning at about 8:30, and then left after 3 o'clock on his way home. He saw the trees and trees. He went home. He didn't see any signs of birds or birds and didn't find anything. He walked right to the local grocery store and discovered a little bi

In [9]:
# Total parameters
total_params = sum(p.numel() for p in model.parameters())
# Trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")


Total parameters: 81912576
Trainable parameters: 81912576
